In [1]:
import pandas as pd
import io
from pathlib import Path

In [6]:
# Carga de Archivos y transformaciones generales
df_ventas_y_volumen = pd.read_csv(f'data/Ventas y Volumen - Maxi.csv', encoding='utf-16', header=1)
df_debitos = pd.read_csv(f'data/Debitos totales por Tienda - Maxi.csv', encoding='utf-16', header=1, sep=',', decimal=',')
padron = pd.read_excel(f'data/padron - agosto.xlsx', header=17) #type:ignore

# Trabajo sobre Ventas y Volumen
#Me quedo unicamente con las columnas importantes
df_ventas_y_volumen = df_ventas_y_volumen[['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Sector', 'Seccion', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']]

#Renombro las columnas
df_ventas_y_volumen.columns = (df_ventas_y_volumen.columns.str.strip().str.lower().str.replace(" ", "_"))
df_ventas_y_volumen.rename(columns={
'ventas_c/impuesto':'venta',
'venta_en_unidades':'volumen'
}, inplace=True)

#Genero una columna para Obtener el ID tienda
df_ventas_y_volumen['numero_operacional'] = df_ventas_y_volumen['punto_operacional'].str.split('-').str[0]

#Me quedo con las columnas necesarias
ventas = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'venta']]
volumen = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'volumen']]

#Quito los NA de las columans de valores
ventas.dropna(subset=['venta'], how='any', inplace=True)
volumen.dropna(subset=['volumen'], how='any', inplace=True)

#Realizo transformaciones para quitar carateres y convertir las columnas a valores numericos
ventas['venta'] = ventas['venta'].str.replace('.','').str.replace(',','.').astype('float')
volumen['volumen'] = volumen['volumen'].str.split(',').str[0].str.replace('.','').astype('int')

#Renombro las columnas con valores de ambos DF
ventas.rename(columns={
'venta':'valores'
}, inplace=True)

volumen.rename(columns={
'volumen':'valores'
}, inplace=True)

#Categorizo los valores tanto de volumne como de Ventas
ventas['categoria'] = 'VCT'
volumen['categoria'] = 'VOL'

#Agrupo las ventas
ventas_agrupado = ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

#Quito Envases del Volumen y Agrupo
volumen_sin_vol = volumen[~volumen['grupo_de_familia'].str.contains('ENVASES')]
volumen_agrupado = volumen_sin_vol.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

# Trabajo sobre Debitos
# Renombro el DF
debitos_agrupados = df_debitos

# Renombro las columnas como corresponden
debitos_agrupados.columns = debitos_agrupados.columns.str.lower().str.replace(' ','_')
debitos_agrupados.rename(columns={
'cant._tickets_por_local':'valores'
}, inplace=True)

# Renombro la columna de Debitos a valores
debitos_agrupados['categoria'] = 'DEB'

# Genero una columna Categorica
debitos_agrupados['numero_operacional'] = debitos_agrupados['punto_operacional'].str.split('-').str[0]

# Genero columna para el ID tienda
debitos_agrupados = debitos_agrupados[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

# Quito nulos numericos de la columna valores
debitos_agrupados.dropna(subset=['valores'], how='any', inplace=True)

# Convierto la columna de valores a su tipo de datos correspondiente
debitos_agrupados['valores'] = debitos_agrupados['valores'].str.replace('.','').astype(int)

# Trabajo sobre el padron
# Selecciono las columnas que me sirven del padron
padron = padron[['N°', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']] #type:ignore

# Cambio de nombres en el padron
padron.columns = (
padron.columns
.str.lower()
.str.strip()
.str.replace(' ', '_', regex=False)
.str.replace('m²', 'm', regex=False)
.str.replace('.2','')
)

# Formateo la fecha para que tenga sentido
padron['fecha_apertura'] = padron['fecha_apertura'].dt.strftime('%d/%m/%Y')

# Cambio el nombre de la columna N por "Numero Operacional"
padron.rename(columns={'n°':'numero_operacional'}, inplace=True)

# Quito los valores nulos utilizando como referencia la columna Numero Operacional, nombre y fecha apertura
padron.dropna(subset=['numero_operacional', 'nombre', 'fecha_apertura', 'ago'], how='any', inplace=True)

# Genero una funcion para convertir los valores de una columna a mayuscula
def maysc(df: pd.DataFrame, columna: str):
    df[columna] = df[columna].str.upper()
#Aplico la formula a la columna del mes comparable para que todos los valores sean en mayuscula

maysc(padron, 'ago')

# Coloco el numero operacional como numero
padron['numero_operacional'] = padron['numero_operacional'].astype(int)

# Concateno todos los df (venta, debito y volumen) y lo joineo con el padron
df = pd.concat([ventas_agrupado, volumen_agrupado, debitos_agrupados])

# Convierto el ID a numero
df['numero_operacional'] = df['numero_operacional'].astype(int)

# Genero el Join del df Agupado con el Padron con el objetivo de quedarme unicamente con aquellas tiendas Comparables
df_join = df.merge(padron, how='left', on='numero_operacional')

# Trabajo sobre Progresiones Total Formato
# Me quedo unicamente con las columnas que me sirven del DF Joineado (ACA TENGO LA SC DEL MES)
df_join = df_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'fecha_apertura', 'fin_de_cierre', 'provincia','categoria', 'valores', 'ago']]

# Filtro unicamente las lineas que sean Superficie Comparable
df_join_sc = df_join[df_join['ago'] == 'SC']

#Renombro la Columna Mes a Fecha para Luego generar la Columna Mes Correspondiente
df_join_sc.rename(columns={
    'mes':'fecha'
}, inplace=True)
df_join_sc['mes'] = df_join_sc['fecha'].str.split(' ').str[0]

#Agrupo el df por categoria teniendo en cuenta el mes, ya que este me servirá luego para limitar el periodo comparable y la superficie comparable
df_acum_formato = df_join_sc.groupby(['año', 'mes', 'direccion', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero un diccionario con los meses y sus valores numericos de forma auxiliar
orden_meses = {"Enero":1, "Febrero":2, "Marzo":3, "Abril":4, "Mayo":5, "Junio":6, "Julio":7, "Agosto":8, "Septiembre":9, "Octubre":10, "Noviembre":11, "Diciembre":12}

#Genero una columna auziliar para ordenar los meses y luego limitar el periodo
df_acum_formato['aux'] = df_acum_formato['mes'].map(orden_meses)

#Limito el periodo del df al mes comparable que quiero
mes_limite = orden_meses['Agosto']
df_acum_formato = df_acum_formato.loc[df_acum_formato['aux'] <= mes_limite]

#Vuelvo a ordenar los meses
df_acum_formato = df_acum_formato.sort_values('aux', ascending=True)

#Una vez que tengo limitado el df por los meses que me interesan, agrupo el df para quitar el detalle de los meses ya que lo que queremos obtener es la sumatoria de los debitos, ventas y volumen del periodo acumulado indicado
df_acum_formato = df_acum_formato.groupby(['categoria'])[[2024, 2025]].sum().reset_index()

#Calculo la Progresion
df_acum_formato['progresion'] = round((((df_acum_formato[2025] / df_acum_formato[2024]) - 1) * 100), 1)

### Trabajo sobre las provincias
#Agrupo el df por categoria teniendo en cuenta el mes, ya que este me servirá luego para limitar el periodo comparable y la superficie comparable
df_acum_provincia = df_join_sc.groupby(['año', 'mes', 'direccion', 'provincia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'categoria', 'provincia'], columns='año', aggfunc='sum').reset_index()

#Genero un diccionario con los meses y sus valores numericos de forma auxiliar
orden_meses = {"Enero":1, "Febrero":2, "Marzo":3, "Abril":4, "Mayo":5, "Junio":6, "Julio":7, "Agosto":8, "Septiembre":9, "Octubre":10, "Noviembre":11, "Diciembre":12}

#Genero una columna auziliar para ordenar los meses y luego limitar el periodo
df_acum_provincia['aux'] = df_acum_provincia['mes'].map(orden_meses)

#Limito el periodo del df al mes comparable que quiero
df_acum_provincia = df_acum_provincia.loc[df_acum_provincia['aux'] <= mes_limite]

# Una vez que tengo el periodo, ya el mes no me sirve, por eso agrupo por provincia y categoria
df_acum_provincia = df_acum_provincia.groupby(['categoria', 'provincia'])[[2024, 2025]].sum().reset_index()

#calculo las Progresiones
df_acum_provincia['progresion'] = round((((df_acum_provincia[2025] / df_acum_provincia[2024]) - 1) * 100), 1)

#Pivot para mostrar mejora la info
df_acum_provincia = df_acum_provincia.pivot_table(values=[2024, 2025, 'progresion'], columns='categoria', index='provincia', aggfunc='sum').sort_values(('progresion', 'VOL'), ascending=False)

### Trabajo sobre las tiendas
df_acum_tiendas = df_join_sc.groupby(['año', 'mes', 'direccion', 'categoria', 'punto_operacional'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'categoria', 'punto_operacional'], columns='año', aggfunc='sum').reset_index()

#Genero un diccionario con los meses y sus valores numericos de forma auxiliar
orden_meses = {"Enero":1, "Febrero":2, "Marzo":3, "Abril":4, "Mayo":5, "Junio":6, "Julio":7, "Agosto":8, "Septiembre":9, "Octubre":10, "Noviembre":11, "Diciembre":12}

#Genero una columna auziliar para ordenar los meses y luego limitar el periodo
df_acum_tiendas['aux'] = df_acum_tiendas['mes'].map(orden_meses)

#Limito el periodo del df al mes comparable que quiero
df_acum_tiendas = df_acum_tiendas.loc[df_acum_tiendas['aux'] <= mes_limite]

#Vuelvo a ordenar los meses
df_acum_tiendas = df_acum_tiendas.sort_values('aux', ascending=True)

#Una vez que tengo limitado el df por los meses que me interesan, agrupo el df para quitar el detalle de los meses ya que lo que queremos obtener es la sumatoria de los debitos, ventas y volumen del periodo acumulado indicado
df_acum_tiendas = df_acum_tiendas.groupby(['punto_operacional', 'categoria'])[[2024, 2025]].sum().reset_index()

#Calculo la Progresion
df_acum_tiendas['progresion'] = round((((df_acum_tiendas[2025] / df_acum_tiendas[2024]) - 1) * 100), 1)
df_acum_tiendas = df_acum_tiendas.pivot_table(values=[2024, 2025, 'progresion'], columns='categoria', index='punto_operacional', aggfunc='sum').sort_values(('progresion', 'VOL'), ascending=False)

### Trabajo con VOL y VCT por Sector, Seccion y GF
#Concateno las ventas con el volumen sin Envases con el objetivo de agruparlo por sus distintas carecteristicas y  asi conseguir las progresiones totales por Sector, seccion y grupo de familia
acumulado_venta_volumen = pd.concat([ventas, volumen_sin_vol])

#Convierto la columna Numero Operacional para realizar el merge con el padron
acumulado_venta_volumen['numero_operacional'] = acumulado_venta_volumen['numero_operacional'].astype(int) 
acumulado_venta_volumen = acumulado_venta_volumen.merge(padron, how='left', on='numero_operacional')

#Me quedo unicamente con las columnas que me sirven y los valores comparables
acumulado_venta_volumen = acumulado_venta_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia','fecha_apertura', 'fin_de_cierre', 'provincia', 'categoria', 'valores', 'ago']]
acumulado_venta_volumen = acumulado_venta_volumen[acumulado_venta_volumen['ago'] == 'SC'] 

#Renomrbo la columna mes a fecha y genero la columna de mes correcta
acumulado_venta_volumen.rename(columns={
    'mes':'fecha'
}, inplace=True)
acumulado_venta_volumen['mes'] = acumulado_venta_volumen['fecha'].str.split(' ').str[0]

#Genero un diccionario con los meses y sus valores numericos de forma auxiliar
orden_meses = {"Enero":1, "Febrero":2, "Marzo":3, "Abril":4, "Mayo":5, "Junio":6, "Julio":7, "Agosto":8, "Septiembre":9, "Octubre":10, "Noviembre":11, "Diciembre":12}

#Genero una columna auziliar para ordenar los meses y luego limitar el periodo
acumulado_venta_volumen['aux'] = acumulado_venta_volumen['mes'].map(orden_meses)

#Limito el periodo del df al mes comparable que quiero
acumulado_venta_volumen = acumulado_venta_volumen.loc[acumulado_venta_volumen['aux'] <= mes_limite]

#Vuelvo a ordenar los meses
acumulado_venta_volumen = acumulado_venta_volumen.sort_values('aux', ascending=True)

#Agrupo por y trabajo por Sector
acumulado_venta_volumen_sector = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'categoria'])['valores'].sum().reset_index()

#Pivoteo la Info para generar las Progresiones
acumulado_venta_volumen_sector = acumulado_venta_volumen_sector.groupby(['año', 'mes', 'direccion', 'sector', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['sector', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero la Progresion
acumulado_venta_volumen_sector['progresion'] = round(((acumulado_venta_volumen_sector[2025] / acumulado_venta_volumen_sector[2024])-1)*100,1)

#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
acumulado_venta_volumen_sector = acumulado_venta_volumen_sector.pivot_table(values=[2024, 2025, 'progresion'], index='sector', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

### Agrupo y Trabajo por Seccion
acumulado_venta_volumen_seccion = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'seccion', 'categoria'])['valores'].sum().reset_index()

#Pivoteo la Info para generar las Progresiones
acumulado_venta_volumen_seccion = acumulado_venta_volumen_seccion.groupby(['año', 'mes', 'direccion', 'seccion', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['seccion', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero la Progresion
acumulado_venta_volumen_seccion['progresion'] = round(((acumulado_venta_volumen_seccion[2025] / acumulado_venta_volumen_seccion[2024])-1)*100,1)

#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
acumulado_venta_volumen_seccion = acumulado_venta_volumen_seccion.pivot_table(values=[2024, 2025, 'progresion'], index='seccion', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

### Agrupo y trabajo por grupo de familia
acumulado_venta_volumen_grupo_de_familia = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index()

#Pivoteo la Info para generar las Progresiones
acumulado_venta_volumen_grupo_de_familia = acumulado_venta_volumen_grupo_de_familia.groupby(['año', 'mes', 'direccion', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['grupo_de_familia', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero la Progresion
acumulado_venta_volumen_grupo_de_familia['progresion'] = round(((acumulado_venta_volumen_grupo_de_familia[2025] / acumulado_venta_volumen_grupo_de_familia[2024])-1)*100,1)

#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
acumulado_venta_volumen_grupo_de_familia = acumulado_venta_volumen_grupo_de_familia.pivot_table(values=[2024, 2025, 'progresion'], index='grupo_de_familia', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

#Agrupo y trabajo por Tienda / Sector
acumulado_venta_volumen_tienda_sector = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'categoria'])['valores'].sum().reset_index()

#Pivoteo la Info para generar las Progresiones
acumulado_venta_volumen_tienda_sector = acumulado_venta_volumen_tienda_sector.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'sector', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['sector', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero la Progresion
acumulado_venta_volumen_tienda_sector['progresion'] = round(((acumulado_venta_volumen_tienda_sector[2025] / acumulado_venta_volumen_tienda_sector[2024])-1)*100,1)

#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
acumulado_venta_volumen_tienda_sector = acumulado_venta_volumen_tienda_sector.pivot_table(values=[2024, 2025, 'progresion'], index=['punto_operacional', 'sector'], columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False).reset_index()

#Agrupo y trabajo por Tienda / Seccion
acumulado_venta_volumen_tienda_seccion = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'seccion', 'categoria'])['valores'].sum().reset_index()

#Pivoteo la Info para generar las Progresiones
acumulado_venta_volumen_tienda_seccion = acumulado_venta_volumen_tienda_seccion.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'seccion', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['seccion', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero la Progresion
acumulado_venta_volumen_tienda_seccion['progresion'] = round(((acumulado_venta_volumen_tienda_seccion[2025] / acumulado_venta_volumen_tienda_seccion[2024])-1)*100,1)

#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
acumulado_venta_volumen_tienda_seccion = acumulado_venta_volumen_tienda_seccion.pivot_table(values=[2024, 2025, 'progresion'], index=['punto_operacional', 'seccion'], columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False).reset_index()

#Agrupo y trabajo por Tienda / GF
acumulado_venta_volumen_tienda_grupo_de_familia = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index()

#Pivoteo la Info para generar las Progresiones
acumulado_venta_volumen_tienda_grupo_de_familia = acumulado_venta_volumen_tienda_grupo_de_familia.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['grupo_de_familia', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero la Progresion
acumulado_venta_volumen_tienda_grupo_de_familia['progresion'] = round(((acumulado_venta_volumen_tienda_grupo_de_familia[2025] / acumulado_venta_volumen_tienda_grupo_de_familia[2024])-1)*100,1)

#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
acumulado_venta_volumen_tienda_grupo_de_familia = acumulado_venta_volumen_tienda_grupo_de_familia.pivot_table(values=[2024, 2025, 'progresion'], index=['punto_operacional', 'grupo_de_familia'], columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False).reset_index()

#Aperturo para dejar toda la informacion lista para que el usuario realice una tabla Pivot y tenga todo de forma  compacta
acumulado_venta_volumen_total = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index()


KeyboardInterrupt



In [365]:
try:
    meses_invertido = {v: k for k, v in orden_meses.items()}
    out_dir = Path("results")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_file = out_dir / f"Resultados Acumulados - {meses_invertido[mes_limite]} - {df_join_sc['direccion'].unique()[0]}.xlsx"

    with pd.ExcelWriter(out_file) as writer:
        df_acum_formato.to_excel(writer, sheet_name=f"Prog Acum {df_join_sc['direccion'].unique()[0]} - SC", index=False)
        df_acum_provincia.to_excel(writer, sheet_name=f"Prog Acum Provincia - SC", index=True)
        df_acum_tiendas.to_excel(writer, sheet_name=f"Prog Acum tiendas - SC", index=True)
        acumulado_venta_volumen_sector.to_excel(writer, sheet_name=f"Prog Acum Sector - SC", index=True)
        acumulado_venta_volumen_seccion.to_excel(writer, sheet_name=f"Prog Acum Seccion - SC", index=True)
        acumulado_venta_volumen_grupo_de_familia.to_excel(writer, sheet_name=f"Prog Acum GF - SC", index=True)
        acumulado_venta_volumen_tienda_sector.to_excel(writer, sheet_name=f"Prog Acum TSect - SC", index=True)
        acumulado_venta_volumen_tienda_seccion.to_excel(writer, sheet_name=f"Prog Acum TSecc - SC", index=True)
        acumulado_venta_volumen_tienda_grupo_de_familia.to_excel(writer, sheet_name=f"Prog Acum TGf - SC", index=True)
        acumulado_venta_volumen_total.to_excel(writer, sheet_name=f"Prog Aperturado x Tienda - SC", index=False)

    print(f"Archivo guardado en: {out_file}")

except Exception as e:
    print(f"No se logró guardar la información en un archivo Excel. ERROR: {e}")

Archivo guardado en: results\Resultados Acumulados - Agosto - MAXI.xlsx


In [7]:
def progresiones_acumulado(mes_comparable:str): 
    try:
        # Carga de Archivos y transformaciones generales
        df_ventas_y_volumen = pd.read_csv('data/ventas maxi.csv', encoding='utf-16', header=1)
        df_debitos = pd.read_csv('data/debitos maxi.csv', encoding='utf-16', header=1, sep=',', decimal=',')
        padron = pd.read_excel('data/padron.xlsx', header=17) #type:ignore

        # Trabajo sobre Ventas y Volumen
        #Me quedo unicamente con las columnas importantes
        df_ventas_y_volumen = df_ventas_y_volumen[['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Sector', 'Seccion', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']]

        #Renombro las columnas
        df_ventas_y_volumen.columns = (df_ventas_y_volumen.columns.str.strip().str.lower().str.replace(" ", "_"))
        df_ventas_y_volumen.rename(columns={
        'ventas_c/impuesto':'venta',
        'venta_en_unidades':'volumen'
        }, inplace=True)

        #Genero una columna para Obtener el ID tienda
        df_ventas_y_volumen['numero_operacional'] = df_ventas_y_volumen['punto_operacional'].str.split('-').str[0]

        #Me quedo con las columnas necesarias
        ventas = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'venta']]
        volumen = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'volumen']]

        #Quito los NA de las columans de valores
        ventas.dropna(subset=['venta'], how='any', inplace=True)
        volumen.dropna(subset=['volumen'], how='any', inplace=True)

        #Realizo transformaciones para quitar carateres y convertir las columnas a valores numericos
        ventas['venta'] = ventas['venta'].str.replace('.','').str.replace(',','.').astype('float')
        volumen['volumen'] = volumen['volumen'].str.split(',').str[0].str.replace('.','').astype('int')

        #Renombro las columnas con valores de ambos DF
        ventas.rename(columns={
        'venta':'valores'
        }, inplace=True)

        volumen.rename(columns={
        'volumen':'valores'
        }, inplace=True)

        #Categorizo los valores tanto de volumne como de Ventas
        ventas['categoria'] = 'VCT'
        volumen['categoria'] = 'VOL'

        #Agrupo las ventas
        ventas_agrupado = ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

        #Quito Envases del Volumen y Agrupo
        volumen_sin_vol = volumen[~volumen['grupo_de_familia'].str.contains('ENVASES')]
        volumen_agrupado = volumen_sin_vol.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

        # Trabajo sobre Debitos
        # Renombro el DF
        debitos_agrupados = df_debitos

        # Renombro las columnas como corresponden
        debitos_agrupados.columns = debitos_agrupados.columns.str.lower().str.replace(' ','_')
        debitos_agrupados.rename(columns={
        'cant._tickets_por_local':'valores'
        }, inplace=True)

        # Renombro la columna de Debitos a valores
        debitos_agrupados['categoria'] = 'DEB'

        # Genero una columna Categorica
        debitos_agrupados['numero_operacional'] = debitos_agrupados['punto_operacional'].str.split('-').str[0]

        # Genero columna para el ID tienda
        debitos_agrupados = debitos_agrupados[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

        # Quito nulos numericos de la columna valores
        debitos_agrupados.dropna(subset=['valores'], how='any', inplace=True)

        # Convierto la columna de valores a su tipo de datos correspondiente
        debitos_agrupados['valores'] = debitos_agrupados['valores'].str.replace('.','').astype(int)

        # Trabajo sobre el padron
        # Selecciono las columnas que me sirven del padron
        padron = padron[['N°', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']] #type:ignore

        # Cambio de nombres en el padron
        padron.columns = (
        padron.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '_', regex=False)
        .str.replace('m²', 'm', regex=False)
        .str.replace('.2','')
        )

        # Formateo la fecha para que tenga sentido
        padron['fecha_apertura'] = padron['fecha_apertura'].dt.strftime('%d/%m/%Y')

        # Cambio el nombre de la columna N por "Numero Operacional"
        padron.rename(columns={'n°':'numero_operacional'}, inplace=True)

        # Quito los valores nulos utilizando como referencia la columna Numero Operacional, nombre y fecha apertura
        padron.dropna(subset=['numero_operacional', 'nombre', 'fecha_apertura', mes_comparable[0:3].lower()], how='any', inplace=True)

        # Genero una funcion para convertir los valores de una columna a mayuscula
        def maysc(df: pd.DataFrame, columna: str):
            df[columna] = df[columna].str.upper()
        #Aplico la formula a la columna del mes comparable para que todos los valores sean en mayuscula

        maysc(padron, mes_comparable[0:3].lower())

        # Coloco el numero operacional como numero
        padron['numero_operacional'] = padron['numero_operacional'].astype(int)

        # Concateno todos los df (venta, debito y volumen) y lo joineo con el padron
        df = pd.concat([ventas_agrupado, volumen_agrupado, debitos_agrupados])

        # Convierto el ID a numero
        df['numero_operacional'] = df['numero_operacional'].astype(int)

        # Genero el Join del df Agupado con el Padron con el objetivo de quedarme unicamente con aquellas tiendas Comparables
        df_join = df.merge(padron, how='left', on='numero_operacional')

        # Trabajo sobre Progresiones Total Formato
        # Me quedo unicamente con las columnas que me sirven del DF Joineado (ACA TENGO LA SC DEL MES)
        df_join = df_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'fecha_apertura', 'fin_de_cierre', 'provincia','categoria', 'valores', mes_comparable[0:3].lower()]]

        # Filtro unicamente las lineas que sean Superficie Comparable
        df_join_sc = df_join[df_join[mes_comparable[0:3].lower()] == 'SC']

        #Renombro la Columna Mes a Fecha para Luego generar la Columna Mes Correspondiente
        df_join_sc.rename(columns={
            'mes':'fecha'
        }, inplace=True)
        df_join_sc['mes'] = df_join_sc['fecha'].str.split(' ').str[0]

        #Agrupo el df por categoria teniendo en cuenta el mes, ya que este me servirá luego para limitar el periodo comparable y la superficie comparable
        df_acum_formato = df_join_sc.groupby(['año', 'mes', 'direccion', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'categoria'], columns='año', aggfunc='sum').reset_index()

        #Genero un diccionario con los meses y sus valores numericos de forma auxiliar
        orden_meses = {"Enero":1, "Febrero":2, "Marzo":3, "Abril":4, "Mayo":5, "Junio":6, "Julio":7, "Agosto":8, "Septiembre":9, "Octubre":10, "Noviembre":11, "Diciembre":12}

        #Genero una columna auziliar para ordenar los meses y luego limitar el periodo
        df_acum_formato['aux'] = df_acum_formato['mes'].map(orden_meses)

        #Limito el periodo del df al mes comparable que quiero
        mes_limite = orden_meses[mes_comparable.capitalize()]
        df_acum_formato = df_acum_formato.loc[df_acum_formato['aux'] <= mes_limite]

        #Vuelvo a ordenar los meses
        df_acum_formato = df_acum_formato.sort_values('aux', ascending=True)

        #Una vez que tengo limitado el df por los meses que me interesan, agrupo el df para quitar el detalle de los meses ya que lo que queremos obtener es la sumatoria de los debitos, ventas y volumen del periodo acumulado indicado
        df_acum_formato = df_acum_formato.groupby(['categoria'])[[2024, 2025]].sum().reset_index()

        #Calculo la Progresion
        df_acum_formato['progresion'] = round((((df_acum_formato[2025] / df_acum_formato[2024]) - 1) * 100), 1)

        ### Trabajo sobre las provincias
        #Agrupo el df por categoria teniendo en cuenta el mes, ya que este me servirá luego para limitar el periodo comparable y la superficie comparable
        df_acum_provincia = df_join_sc.groupby(['año', 'mes', 'direccion', 'provincia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'categoria', 'provincia'], columns='año', aggfunc='sum').reset_index()

        #Genero una columna auziliar para ordenar los meses y luego limitar el periodo
        df_acum_provincia['aux'] = df_acum_provincia['mes'].map(orden_meses)

        #Limito el periodo del df al mes comparable que quiero
        df_acum_provincia = df_acum_provincia.loc[df_acum_provincia['aux'] <= mes_limite]

        # Una vez que tengo el periodo, ya el mes no me sirve, por eso agrupo por provincia y categoria
        df_acum_provincia = df_acum_provincia.groupby(['categoria', 'provincia'])[[2024, 2025]].sum().reset_index()

        #calculo las Progresiones
        df_acum_provincia['progresion'] = round((((df_acum_provincia[2025] / df_acum_provincia[2024]) - 1) * 100), 1)

        #Pivot para mostrar mejora la info
        df_acum_provincia = df_acum_provincia.pivot_table(values=[2024, 2025, 'progresion'], columns='categoria', index='provincia', aggfunc='sum').sort_values(('progresion', 'VOL'), ascending=False)

        ### Trabajo sobre las tiendas
        df_acum_tiendas = df_join_sc.groupby(['año', 'mes', 'direccion', 'categoria', 'punto_operacional'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'categoria', 'punto_operacional'], columns='año', aggfunc='sum').reset_index()

        #Genero una columna auziliar para ordenar los meses y luego limitar el periodo
        df_acum_tiendas['aux'] = df_acum_tiendas['mes'].map(orden_meses)

        #Limito el periodo del df al mes comparable que quiero
        df_acum_tiendas = df_acum_tiendas.loc[df_acum_tiendas['aux'] <= mes_limite]

        #Vuelvo a ordenar los meses
        df_acum_tiendas = df_acum_tiendas.sort_values('aux', ascending=True)

        #Una vez que tengo limitado el df por los meses que me interesan, agrupo el df para quitar el detalle de los meses ya que lo que queremos obtener es la sumatoria de los debitos, ventas y volumen del periodo acumulado indicado
        df_acum_tiendas = df_acum_tiendas.groupby(['punto_operacional', 'categoria'])[[2024, 2025]].sum().reset_index()

        #Calculo la Progresion
        df_acum_tiendas['progresion'] = round((((df_acum_tiendas[2025] / df_acum_tiendas[2024]) - 1) * 100), 1)
        df_acum_tiendas = df_acum_tiendas.pivot_table(values=[2024, 2025, 'progresion'], columns='categoria', index='punto_operacional', aggfunc='sum').sort_values(('progresion', 'VOL'), ascending=False)

        ### Trabajo con VOL y VCT por Sector, Seccion y GF
        #Concateno las ventas con el volumen sin Envases con el objetivo de agruparlo por sus distintas carecteristicas y  asi conseguir las progresiones totales por Sector, seccion y grupo de familia
        acumulado_venta_volumen = pd.concat([ventas, volumen_sin_vol])

        #Convierto la columna Numero Operacional para realizar el merge con el padron
        acumulado_venta_volumen['numero_operacional'] = acumulado_venta_volumen['numero_operacional'].astype(int) 
        acumulado_venta_volumen = acumulado_venta_volumen.merge(padron, how='left', on='numero_operacional')

        #Me quedo unicamente con las columnas que me sirven y los valores comparables
        acumulado_venta_volumen = acumulado_venta_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia','fecha_apertura', 'fin_de_cierre', 'provincia', 'categoria', 'valores', mes_comparable[0:3].lower()]]
        acumulado_venta_volumen = acumulado_venta_volumen[acumulado_venta_volumen[mes_comparable[0:3].lower()] == 'SC'] 

        #Renomrbo la columna mes a fecha y genero la columna de mes correcta
        acumulado_venta_volumen.rename(columns={
            'mes':'fecha'
        }, inplace=True)
        acumulado_venta_volumen['mes'] = acumulado_venta_volumen['fecha'].str.split(' ').str[0]

        #Genero una columna auziliar para ordenar los meses y luego limitar el periodo
        acumulado_venta_volumen['aux'] = acumulado_venta_volumen['mes'].map(orden_meses)

        #Limito el periodo del df al mes comparable que quiero
        acumulado_venta_volumen = acumulado_venta_volumen.loc[acumulado_venta_volumen['aux'] <= mes_limite]

        #Vuelvo a ordenar los meses
        acumulado_venta_volumen = acumulado_venta_volumen.sort_values('aux', ascending=True)

        #Agrupo por y trabajo por Sector
        acumulado_venta_volumen_sector = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'categoria'])['valores'].sum().reset_index()

        #Pivoteo la Info para generar las Progresiones
        acumulado_venta_volumen_sector = acumulado_venta_volumen_sector.groupby(['año', 'mes', 'direccion', 'sector', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['sector', 'categoria'], columns='año', aggfunc='sum').reset_index()

        #Genero la Progresion
        acumulado_venta_volumen_sector['progresion'] = round(((acumulado_venta_volumen_sector[2025] / acumulado_venta_volumen_sector[2024])-1)*100,1)

        #Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
        acumulado_venta_volumen_sector = acumulado_venta_volumen_sector.pivot_table(values=[2024, 2025, 'progresion'], index='sector', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

        ### Agrupo y Trabajo por Seccion
        acumulado_venta_volumen_seccion = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'seccion', 'categoria'])['valores'].sum().reset_index()

        #Pivoteo la Info para generar las Progresiones
        acumulado_venta_volumen_seccion = acumulado_venta_volumen_seccion.groupby(['año', 'mes', 'direccion', 'seccion', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['seccion', 'categoria'], columns='año', aggfunc='sum').reset_index()

        #Genero la Progresion
        acumulado_venta_volumen_seccion['progresion'] = round(((acumulado_venta_volumen_seccion[2025] / acumulado_venta_volumen_seccion[2024])-1)*100,1)

        #Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
        acumulado_venta_volumen_seccion = acumulado_venta_volumen_seccion.pivot_table(values=[2024, 2025, 'progresion'], index='seccion', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

        ### Agrupo y trabajo por grupo de familia
        acumulado_venta_volumen_grupo_de_familia = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index()

        #Pivoteo la Info para generar las Progresiones
        acumulado_venta_volumen_grupo_de_familia = acumulado_venta_volumen_grupo_de_familia.groupby(['año', 'mes', 'direccion', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['grupo_de_familia', 'categoria'], columns='año', aggfunc='sum').reset_index()

        #Genero la Progresion
        acumulado_venta_volumen_grupo_de_familia['progresion'] = round(((acumulado_venta_volumen_grupo_de_familia[2025] / acumulado_venta_volumen_grupo_de_familia[2024])-1)*100,1)

        #Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
        acumulado_venta_volumen_grupo_de_familia = acumulado_venta_volumen_grupo_de_familia.pivot_table(values=[2024, 2025, 'progresion'], index='grupo_de_familia', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

        #Agrupo y trabajo por Tienda / Sector
        acumulado_venta_volumen_tienda_sector = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'categoria'])['valores'].sum().reset_index()

        #Pivoteo la Info para generar las Progresiones
        acumulado_venta_volumen_tienda_sector = acumulado_venta_volumen_tienda_sector.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'sector', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['sector', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()

        #Genero la Progresion
        acumulado_venta_volumen_tienda_sector['progresion'] = round(((acumulado_venta_volumen_tienda_sector[2025] / acumulado_venta_volumen_tienda_sector[2024])-1)*100,1)

        #Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
        acumulado_venta_volumen_tienda_sector = acumulado_venta_volumen_tienda_sector.pivot_table(values=[2024, 2025, 'progresion'], index=['punto_operacional', 'sector'], columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False).reset_index()

        #Agrupo y trabajo por Tienda / Seccion
        acumulado_venta_volumen_tienda_seccion = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'seccion', 'categoria'])['valores'].sum().reset_index()

        #Pivoteo la Info para generar las Progresiones
        acumulado_venta_volumen_tienda_seccion = acumulado_venta_volumen_tienda_seccion.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'seccion', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['seccion', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()

        #Genero la Progresion
        acumulado_venta_volumen_tienda_seccion['progresion'] = round(((acumulado_venta_volumen_tienda_seccion[2025] / acumulado_venta_volumen_tienda_seccion[2024])-1)*100,1)

        #Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
        acumulado_venta_volumen_tienda_seccion = acumulado_venta_volumen_tienda_seccion.pivot_table(values=[2024, 2025, 'progresion'], index=['punto_operacional', 'seccion'], columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False).reset_index()

        #Agrupo y trabajo por Tienda / GF
        acumulado_venta_volumen_tienda_grupo_de_familia = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index()

        #Pivoteo la Info para generar las Progresiones
        acumulado_venta_volumen_tienda_grupo_de_familia = acumulado_venta_volumen_tienda_grupo_de_familia.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['grupo_de_familia', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()

        #Genero la Progresion
        acumulado_venta_volumen_tienda_grupo_de_familia['progresion'] = round(((acumulado_venta_volumen_tienda_grupo_de_familia[2025] / acumulado_venta_volumen_tienda_grupo_de_familia[2024])-1)*100,1)

        #Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
        acumulado_venta_volumen_tienda_grupo_de_familia = acumulado_venta_volumen_tienda_grupo_de_familia.pivot_table(values=[2024, 2025, 'progresion'], index=['punto_operacional', 'grupo_de_familia'], columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False).reset_index()

        #Aperturo para dejar toda la informacion lista para que el usuario realice una tabla Pivot y tenga todo de forma  compacta
        acumulado_venta_volumen_total = acumulado_venta_volumen.groupby(['año', 'mes', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index()

        try:
            meses_invertido = {v: k for k, v in orden_meses.items()}
            out_dir = Path("results")
            out_dir.mkdir(parents=True, exist_ok=True)
            out_file = out_dir / f"Resultados Acumulados - {meses_invertido[mes_limite]} - {df_join_sc['direccion'].unique()[0]}.xlsx"

            with pd.ExcelWriter(out_file) as writer:
                df_acum_formato.to_excel(writer, sheet_name=f"Prog Acum {df_join_sc['direccion'].unique()[0]} - SC", index=False)
                df_acum_provincia.to_excel(writer, sheet_name=f"Prog Acum Provincia - SC", index=True)
                df_acum_tiendas.to_excel(writer, sheet_name=f"Prog Acum tiendas - SC", index=True)
                acumulado_venta_volumen_sector.to_excel(writer, sheet_name=f"Prog Acum Sector - SC", index=True)
                acumulado_venta_volumen_seccion.to_excel(writer, sheet_name=f"Prog Acum Seccion - SC", index=True)
                acumulado_venta_volumen_grupo_de_familia.to_excel(writer, sheet_name=f"Prog Acum GF - SC", index=True)
                acumulado_venta_volumen_tienda_sector.to_excel(writer, sheet_name=f"Prog Acum TSect - SC", index=True)
                acumulado_venta_volumen_tienda_seccion.to_excel(writer, sheet_name=f"Prog Acum TSecc - SC", index=True)
                acumulado_venta_volumen_tienda_grupo_de_familia.to_excel(writer, sheet_name=f"Prog Acum TGf - SC", index=True)
                acumulado_venta_volumen_total.to_excel(writer, sheet_name=f"Prog Aperturado x Tienda - SC", index=False)

            print(f"Archivo guardado en: {out_file}")

        except Exception as e:
            print(f"No se logró guardar la información en un archivo Excel. ERROR: {e}")

    except Exception as e:
        print(e)
        return None

In [8]:
progresiones_acumulado('Agosto')

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_8816\239763935.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ventas.dropna(subset=['venta'], how='any', inplace=True)
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_8816\239763935.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volumen.dropna(subset=['volumen'], how='any', inplace=True)
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_8816\239763935.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

Archivo guardado en: results\Resultados Acumulados - Agosto - MAXI.xlsx
